In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Flatten, LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import os


In [74]:
data_vader = pd.read_csv("./sentiment/data_vader.csv", parse_dates = True, index_col = 'Date')

In [75]:
data_vader

,Open,High,Low,Close,biden,musk,trump,tim,bill,jeff
Date,,,,,,,,,,
2017-02-01,2285.590088,2289.139893,2272.439941,2279.550049,0.04150,0.401900,-0.528233,0.8316,0.8834,0.4926
2017-02-02,2276.689941,2283.969971,2271.649902,2280.850098,0.97810,0.401900,0.112517,0.8316,0.8834,0.4926
2017-02-03,2288.540039,2298.310059,2287.879883,2297.419922,0.08900,0.000000,-0.169000,0.8316,0.8316,0.0000
2017-02-06,2294.280029,2296.179932,2288.570068,2292.560059,0.08900,0.169778,-0.425867,0.8316,0.8316,0.0000
2017-02-07,2295.870117,2299.399902,2290.159912,2293.080078,-0.62490,0.169778,0.134033,0.8316,0.8316,0.0000
...,...,...,...,...,...,...,...,...,...,...
2020-12-23,3693.419922,3711.239990,3689.280029,3690.010010,0.12035,-0.018150,-0.073100,0.0000,0.0000,0.3612
2020-12-24,3694.030029,3703.820068,3689.320068,3703.060059,0.12035,0.198250,-0.170190,0.0000,0.0000,0.3612
2020-12-28,3723.030029,3740.510010,3723.030029,3735.360107,0.12035,0.123214,0.481933,0.0000,0.0000,0.3612


In [76]:
scalar = MinMaxScaler()

In [78]:
data = {}
for column_name in data_vader.columns[4:]:
    cols_to_keep = ['Open', 'High', 'Low', 'Close']
    cols_to_keep.append(column_name)
    data.update({column_name : data_vader[cols_to_keep]})

In [79]:
scaled = {}
for i in data.keys():
    scaled.update({ i : scalar.fit_transform(data[i])})

In [80]:
def get_train_values(j):
    train_ind = int(0.6*len(data[j]))
    val_ind = train_ind + int(0.2*len(data[j]))
    train = scaled[j][:train_ind]
    val = scaled[j][train_ind:val_ind]
    test = scaled[j][val_ind:]
    xtrain,ytrain,xval,yval,xtest,ytest = train[:,:],train[:,3],val[:,:],val[:,3],test[:,:],test[:,3]
    lookback = 1
    n_features = 5
    train_len = len(xtrain) - lookback
    test_len = len(xtest) - lookback
    val_len = len(xval) - lookback
    x_train = np.zeros((train_len, lookback, n_features))
    y_train = np.zeros((train_len))
    for i in range(train_len):
        ytemp = i+lookback
        x_train[i] = xtrain[i:ytemp]
        y_train[i] = ytrain[ytemp]
    x_val = np.zeros((val_len, lookback, n_features))
    y_val = np.zeros((val_len))
    for i in range(val_len):
        ytemp = i+lookback
        x_val[i] = xval[i:ytemp]
        y_val[i] = yval[ytemp]
    x_test = np.zeros((test_len, lookback, n_features))
    y_test = np.zeros((test_len))
    for i in range(test_len):
        ytemp = i+lookback
        x_test[i] = xtest[i:ytemp]
        y_test[i] = ytest[ytemp]
    return x_train, y_train, x_test, y_test

In [81]:
def build_and_test_model(name):
    print(f"Model {name}")
    model = Sequential()
    model.add(LSTM(120,input_shape = (lookback, n_features), return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = 'adam')
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10,  verbose=1, mode='min', restore_best_weights=True, start_from_epoch=10)
    model.summary()
    start = time()
    print("start:",0)
    model.fit(x_train,y_train, epochs = 50, batch_size=8,
              validation_data=(x_test,y_test),verbose = 1,
              shuffle = False, callbacks=[earlystop])
    print("end:",time()-start)
    return model

In [82]:
def loss_view(model, i):
    target_directory = f'./models_with_sentiment/loss'
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    loss = model.history.history  # וודא שהמשתנה הזה מכיל את המידע הרצוי
    plt.figure(figsize=(10, 6))
    plt.plot(loss['loss'], label='Training Loss', linestyle='-', linewidth=2, color='b')
    plt.plot(loss['val_loss'], label='Validation Loss', linestyle='--', linewidth=2, color='r')
    plt.title('Loss Over Time')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)  # הוסף גריד על הגרף
    plt.tight_layout()  # משפר את הפריסה של הגרף
    plt.savefig(os.path.join(target_directory, f"Model_{i}.png"), transparent=True)
    plt.show()

In [83]:
def prediction(model,i):
    target_directory = f'./models_with_sentiment/prediction'
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    y_pred = model.predict(x_test)
    plt.figure(figsize=(20,10))
    plt.plot( y_test, '.-', color='red', label='Real values', alpha=0.5)
    plt.plot( y_pred, '.-', color='blue', label='Predicted values', alpha=1)
    plt.savefig(os.path.join(target_directory, f"Model_{i}.png"), transparent=True)
    plt.show()
    return y_pred

In [84]:
def scores(y_pred, i):
    r2 = r2_score(y_test, y_pred)
    print(f'R2 Score: {r2}')
    mae = mean_absolute_error(y_test, y_pred)
    print(f'MAE: {mae}')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f'RMSE: {rmse}')
    if os.path.exists("results.xlsx"):
        df = pd.read_excel("results.xlsx", engine='openpyxl')
        df[f"model {i} with sentiment"] = [r2,mae,rmse,lookback]

        df.to_excel("results.xlsx", index=False, engine='openpyxl')

    else:
        # If the file doesn't exist, create a new DataFrame
        df = pd.DataFrame({
            "Metrics": ['R2 Score', 'MAE', 'RMSE', 'lookback'],
            f"model {i} with sentiment": [r2,mae,rmse,lookback]
        })

        # Save the DataFrame to a new XLSX file
        df.to_excel("results.xlsx", index=False, engine='openpyxl')

In [85]:
def save(model, i):
    target_directory = f'./models_with_sentiment/models'
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    model.save(os.path.join(target_directory, f"model_{i}.keras"))

In [ ]:
models_list = {}
for i in data.keys():
    x_train, y_train, x_test, y_test = get_train_values(i)
    models_list[i] = build_and_test_model(i)
    y_pred = prediction(models_list[i], i)
    models_list[i].evaluate(x_test, y_test)
    scores(y_pred, i)
    save(models_list[i], i)